In [ ]:
import shutil
import os

folder_path = '/kaggle/working/cropped'  # Replace with your folder path

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' has been deleted.")
else:
    print(f"Folder '{folder_path}' does not exist.")


In [ ]:
#!pip install dlib imageio imutils

In [ ]:
#Installing Dependencies

In [ ]:
!pip install dlib
!pip install imutils
!pip install matplotlib

In [ ]:
%matplotlib inline

In [ ]:
#Defining Binding Boxes For the input 

In [ ]:
#Video Processing Logic

In [ ]:
def process_video(video_path, output_dir):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('/kaggle/input/lipread/shape_predictor_68_face_landmarks.dat')
    
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = imutils.resize(frame, width=500)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 1)
        
        for (i, rect) in enumerate(rects):
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            (x, y, w, h) = cv2.boundingRect(np.array([shape[48:68]]))
            roi = gray[y:y+h, x:x+w]
            roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
            
            # Save ROI with the frame count
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            cv2.imwrite(f"{output_dir}/frame_{frame_count}.png", roi)
        
        frame_count += 1
    cap.release()


In [ ]:
#Image Cropping Logic

In [ ]:
!pip install dlib
!pip install imutils
!pip install matplotlib
%matplotlib inline

In [1]:
import shutil
import os
import time
import numpy as np
import imutils
import dlib
import cv2
import imageio
from imutils import face_utils
import matplotlib.pyplot as plt
#from skimage.transform import resize
from keras.utils import to_categorical
from sklearn.utils import shuffle
from tensorflow.keras.layers import Conv3D, Dense, Dropout, Flatten, MaxPooling3D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

ImportError: DLL load failed while importing _multiarray_umath: The specified module could not be found.

ImportError: numpy._core.multiarray failed to import

In [2]:
# Directory paths
folder_path = 'E:\Projects\Computer Vision\lipread(MIRACL)\dataset\cropped\cropped'  # Replace with your folder path

# Remove the existing folder if it exists
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Folder '{folder_path}' has been deleted.")
else:
    print(f"Folder '{folder_path}' does not exist.")


Folder 'E:\Projects\Computer Vision\lipread(MIRACL)\dataset\cropped\cropped' has been deleted.


In [ ]:
# Create the cropped directory
#os.mkdir('cropped')


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'cropped'

In [3]:
# Model parameters
max_seq_length = 22
MAX_WIDTH = 100
MAX_HEIGHT = 100

# People, words, and instances setup
people = ['F01', 'F02', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11', 'M01', 'M02', 'M04', 'M07', 'M08']
data_types = ['words']
folder_enum = ['01', '02', '03', '04', '05']
instances = ['01', '02']
words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']


In [4]:
# Ensure directories for cropping
def ensure_directories_exist():
    for person_ID in people:
        person_dir = os.path.join(folder_path, person_ID)
        if not os.path.exists(person_dir):
            os.makedirs(person_dir)

        for data_type in data_types:
            for phrase_ID in folder_enum:
                for instance_ID in instances:
                    path = os.path.join(person_dir, data_type, phrase_ID, instance_ID)
                    if not os.path.exists(path):
                        os.makedirs(path)


In [5]:
# Function to crop the face region and save the image
def crop_and_save_image(img_path, write_img_path, img_name):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('E:\Projects\Computer Vision\lipread(MIRACL)\dataset\shape_predictor_68_face_landmarks.dat')

    image = cv2.imread(img_path)
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 1)
    if len(rects) > 1:
        print("ERROR: more than one face detected")
        return
    if len(rects) < 1:
        print("ERROR: no faces detected")
        return

    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        (x, y, w, h) = cv2.boundingRect(np.array([shape[48:68]]))
        roi = gray[y:y + h, x:x + w]
        roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)

        if not os.path.exists('cropped/' + write_img_path):
            os.makedirs('cropped/' + write_img_path)
        cv2.imwrite('cropped/' + write_img_path + '/' + img_name, roi)

        # Visualization of the transformations
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.title("Original Image")
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.axis("off")

        plt.subplot(1, 2, 2)
        plt.title("Cropped ROI (Mouth)")
        plt.imshow(roi, cmap='gray')
        plt.axis("off")
        plt.show()

In [6]:
# Function to crop images for each person
def crop_one_person():
    for person_ID in people:
        for data_type in data_types:
            for phrase_ID in folder_enum:
                for instance_ID in instances:
                    directory = os.path.join(folder_path, person_ID, data_type, phrase_ID, instance_ID)
                    filelist = os.listdir(directory)

                    for img_name in filelist:
                        if img_name.startswith('color'):
                            crop_and_save_image(os.path.join(directory, img_name), 
                                                 os.path.join(person_ID, data_type, phrase_ID, instance_ID), img_name)

In [7]:
# Running the cropping logic
start_time = time.time()
ensure_directories_exist()
crop_one_person()
end_time = time.time()

print(f"Time taken for image cropping: {end_time - start_time} seconds")

Time taken for image cropping: 0.10399866104125977 seconds


In [8]:
# Load and preprocess images for training
def load_and_preprocess_images():
    X_train, y_train = [], []
    X_val, y_val = [], []
    X_test, y_test = [], []

    UNSEEN_VALIDATION_SPLIT = ['F07', 'M02']
    UNSEEN_TEST_SPLIT = ['F04', 'M01']

    directory = "E:\Projects\Computer Vision\lipread(MIRACL)\dataset\cropped\cropped"

    for person_id in people:
        for data_type in data_types:
            for word_index, word in enumerate(folder_enum):
                for iteration in instances:
                    path = os.path.join(directory, person_id, data_type, word, iteration)
                    filelist = sorted(os.listdir(path + '/'))
                    sequence = [] 
                    for img_name in filelist:
                        if img_name.startswith('color'):
                            image = imageio.imread(path + '/' + img_name)
                            image = resize(image, (MAX_WIDTH, MAX_HEIGHT))
                            image = 255 * image
                            image = image.astype(np.uint8)
                            sequence.append(image)

                    pad_array = [np.zeros((MAX_WIDTH, MAX_HEIGHT))]                            
                    sequence.extend(pad_array * (max_seq_length - len(sequence)))
                
                    if person_id in UNSEEN_TEST_SPLIT:
                        X_test.append(sequence)
                        y_test.append(word_index)
                    elif person_id in UNSEEN_VALIDATION_SPLIT:
                        X_val.append(sequence)
                        y_val.append(word_index)
                    else:
                        X_train.append(sequence)
                        y_train.append(word_index)
    
    return np.array(X_train), np.array(X_val), np.array(X_test), np.array(y_train), np.array(y_val), np.array(y_test)

In [9]:
# Load and preprocess the data
X_train, X_val, X_test, y_train, y_val, y_test = load_and_preprocess_images()

# Normalize the data
def normalize_it(X):
    v_min = X.min(axis=(2, 3), keepdims=True)
    v_max = X.max(axis=(2, 3), keepdims=True)
    range_values = np.where((v_max - v_min) == 0, np.finfo(float).eps, v_max - v_min)
    X_normalized = np.where(range_values == 0, 0, (X - v_min) / range_values)
    return X_normalized

X_train = normalize_it(X_train)
X_val = normalize_it(X_val)
X_test = normalize_it(X_test)


In [10]:
# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

# Shuffle the data
X_train, y_train = shuffle(X_train, y_train, random_state=0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)
X_val, y_val = shuffle(X_val, y_val, random_state=0)

# Add an extra dimension for the grayscale channel
X_train = np.expand_dims(X_train, axis=4)
X_val = np.expand_dims(X_val, axis=4) 
X_test = np.expand_dims(X_test, axis=4)


In [ ]:

# Build the model
model = Sequential()
 
model.add(Conv3D(128, (3, 3, 3), strides=1, input_shape=(22, 100, 100, 1), activation='relu', padding='valid'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model.add(Conv3D(64, (3, 3, 3), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

'''model.add(Conv3D(32, (2, 2, 2), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))'''

model.add(Conv3D(32, (1, 1, 1), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(1, 1, 1), strides=2))


model.add(Flatten())

# Fully connected layers
#model.add(Dense(1024, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()
earlystop=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True )

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[earlystop])



e:\Projects\Computer Vision\lipread(MIRACL)\lipread\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_3 (Conv3D)               │ (None, 20, 98, 98,     │         3,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_3 (MaxPooling3D)  │ (None, 10, 49, 49,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_4 (Conv3D)               │ (None, 8, 47, 47, 64)  │       221,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_4 (MaxPooling3D)  │ (None, 4, 23, 23, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_5 (Conv3D)               │ (None, 4, 23, 23, 32)  │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling3d_5 (MaxPooling3D)  │ (None, 2, 12, 12, 32)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │     1,179,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,415,594 (5.40 MB)

 Trainable params: 1,415,594 (5.40 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100


ValueError: Could not interpret loss identifier: Categorical_crossentropy

In [ ]:
# Save only the weights of the model with the correct file extension
model.save_weights('lip_reading_weights.weights.h5')
print("Model weights saved as 'lip_reading_weights.weights.h5'")

In [ ]:
# Recreate your model architecture as you did before
model = Sequential()

# 1st layer group
model.add(Conv3D(64, (3, 3, 3), strides=1, input_shape=(22, 100, 100, 1), activation='relu', padding='valid'))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model.add(Conv3D(128, (3, 3, 3), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model.add(Conv3D(256, (2, 2, 2), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=2))

model.add(Conv3D(512, (1, 1, 1), activation='relu', strides=1))
model.add(MaxPooling3D(pool_size=(1, 1, 1), strides=2))

model.add(Flatten())

# Fully connected layers
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adagrad', metrics=['accuracy'])

# Now load the weights into the model
model.load_weights('lip_reading_weights.weights.h5')
print("Model weights loaded successfully.")


In [ ]:
import shutil
import os
import numpy as np
import imageio
from skimage.transform import resize
from tensorflow.keras.models import load_model
import dlib
import cv2
import imutils
from imutils import face_utils
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle

# Set constants
MAX_WIDTH = 100
MAX_HEIGHT = 100
max_seq_length = 22  # Maximum sequence length used during training
words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']  # Corresponding to 0-9

# Helper function to preprocess the sequence of images (same as training)
def preprocess_sequence(image_folder):
    filelist = sorted(os.listdir(image_folder))
    sequence = []
    
    for img_name in filelist:
        if img_name.startswith('color'):  # Assuming images start with 'color'
            image = imageio.imread(os.path.join(image_folder, img_name))
            image = resize(image, (MAX_WIDTH, MAX_HEIGHT))  # Resize to (100, 100)
            image = 255 * image  # Scale the image to 255 (similar to training)
            image = image.astype(np.uint8)
            sequence.append(image)

    # Pad sequence if it is shorter than max_seq_length
    pad_array = [np.zeros((MAX_WIDTH, MAX_HEIGHT))]
    sequence.extend(pad_array * (max_seq_length - len(sequence)))
    
    # Convert to numpy array and normalize it
    sequence = np.array(sequence)
    sequence = np.expand_dims(sequence, axis=-1)  # Add channel dimension (grayscale)
    sequence = normalize_it(sequence)  # Normalize
    return sequence

# Normalize function (same as during training)
def normalize_it(X):
    v_min = X.min(axis=(2, 3), keepdims=True)
    v_max = X.max(axis=(2, 3), keepdims=True)
    range_values = np.where((v_max - v_min) == 0, np.finfo(float).eps, v_max - v_min)
    X_normalized = np.where(range_values == 0, 0, (X - v_min) / range_values)
    return X_normalized

# Load your trained model (make sure the model is already trained and saved)
model = load_model("lip_reading_model.h5")

# Helper function to preprocess and predict a sequence
def make_prediction(image_folder):
    # Preprocess the sequence of images
    sequence = preprocess_sequence(image_folder)
    
    # Make the prediction
    predicted_class = model.predict(np.expand_dims(sequence, axis=0))  # Add batch dimension
    predicted_index = np.argmax(predicted_class)  # Get the index of the class with the highest probability
    
    # Map the predicted index to the corresponding word
    predicted_word = words[predicted_index]
    
    # Return the prediction
    return predicted_word

# Example Usage - Test Prediction
image_folder = '/kaggle/working/cropped/F01/words/01/01'  # Path to the folder containing the sequence of images (change to your test folder)
predicted_word = make_prediction(image_folder)

# Print the prediction
print(f"Predicted word: {predicted_word}")

# Load your training data (ensure this part is executed as before)
# Loading the dataset variables
people = ['F01', 'F02', 'F04', 'F05', 'F06', 'F07', 'F08', 'F09', 'F10', 'F11', 'M01', 'M02', 'M04', 'M07', 'M08']
data_types = ['words']
folder_enum = ['01', '02', '03', '04', '05']
instances = ['01', '02']
words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']
words_di = {i: words[i] for i in range(len(words))}

# Image Cropping and Loading Data
def crop_and_save_image(img, img_path, write_img_path, img_name):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('/kaggle/input/lipread/shape_predictor_68_face_landmarks.dat')

    image = cv2.imread(img_path)
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 1)
    if len(rects) > 1:
        print("ERROR: more than one face detected")
        return
    if len(rects) < 1:
        print("ERROR: no faces detected")
        return

    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        (x, y, w, h) = cv2.boundingRect(np.array([shape[48:68]]))
        roi = gray[y:y+h, x:x+w]
        roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)

        if not os.path.exists('cropped/' + write_img_path):
            os.makedirs('cropped/' + write_img_path)
        cv2.imwrite('cropped/' + write_img_path + '/' + img_name, roi)

# Image preprocessing for training
def load_and_preprocess_images():
    X_train = []
    y_train = []
    X_val = []
    y_val = []
    X_test = []
    y_test = []

    directory = "/kaggle/working/cropped/"
    for person_id in people:
        for data_type in data_types:
            for word_index, word in enumerate(folder_enum):
                for iteration in instances:
                    path = os.path.join(directory, person_id, data_type, word, iteration)
                    filelist = sorted(os.listdir(path + '/'))
                    sequence = [] 
                    for img_name in filelist:
                        if img_name.startswith('color'):
                            image = imageio.imread(path + '/' + img_name)
                            image = resize(image, (MAX_WIDTH, MAX_HEIGHT))
                            image = 255 * image
                            image = image.astype(np.uint8)
                            sequence.append(image)                        
                    pad_array = [np.zeros((MAX_WIDTH, MAX_HEIGHT))]                            
                    sequence.extend(pad_array * (max_seq_length - len(sequence)))

                    if person_id in ['F07', 'M02']:
                        X_test.append(sequence)
                        y_test.append(word_index)
                    elif person_id in ['F04', 'M01']:
                        X_val.append(sequence)
                        y_val.append(word_index)
                    else:
                        X_train.append(sequence)
                        y_train.append(word_index)
    
    # Convert to numpy arrays and normalize
    X_train = np.array(X_train)
    X_val = np.array(X_val)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_val = np.array(y_val)
    y_test = np.array(y_test)

    X_train = normalize_it(X_train)
    X_val = normalize_it(X_val)
    X_test = normalize_it(X_test)

    y_train = to_categorical(y_train, 10)
    y_val = to_categorical(y_val, 10)
    y_test = to_categorical(y_test, 10)

    return X_train, X_val, X_test, y_train, y_val, y_test

# Model building (as previously provided)
# Assuming you've trained the model and saved it as "lip_reading_model.h5" already.

# For predictions, make sure to call the function `make_prediction(image_folder)` where `image_folder` is the path to your test folder.

# Now, you can test it with any folder by providing the correct image sequence for a given word.
